In [3]:
import random
import torch
import torch.nn as nn
import numpy as np
import torchmetrics
from typing import Counter
from torchvision.datasets import ImageFolder
from sklearn.utils.class_weight import compute_class_weight
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

# from lightning_datamodule import EmotionDataModule
# from models.lightning_emotion_vgg import EmotionVGG

import os

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

train_dir = '../data/train'
test_dir = '../data/test'

In [4]:
dataset = ImageFolder(train_dir)
class_counts = dict(Counter(dataset.targets))
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.targets), y=dataset.targets)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

In [5]:
class_weights

tensor([1.0266, 9.4066, 1.0010, 0.5684, 0.8260, 0.8491, 1.2934])

In [ ]:
config = {
    'num_classes': 7,
    'batch_norm': True,
    'batch_size': 64,
    'learning_rate': 0.001,
    'epochs': 3,
    'train_dir': train_dir,
    'test_dir': test_dir,
    'metric':torchmetrics.AUROC(num_classes=7, task='multiclass'),
    'loss_fn': nn.CrossEntropyLoss(weight=class_weights),
    'in_channels': 3
}

In [6]:
import wandb
WANDB_API_KEY = os.environ.get('WANDB_API_KEY')
wandb.login(key=WANDB_API_KEY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: serhiikravtsov. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\serhii.kravtsov\.netrc


True

In [7]:
def run_train(config):
    run_name = f"tinyvgg_{config['batch_size']}_{config['learning_rate']}_{config['epochs']}"
    wandb.init(project='emotion-recognition', name=run_name, config=config)
    wandb_logger = WandbLogger()
    model = EmotionVGG(config)
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='../models',
        filename='emotion_classification_model',
        mode='min',
    )
    trainer = Trainer(max_epochs=config['epochs'], logger=wandb_logger, enable_progress_bar=True,
        enable_checkpointing = True, callbacks=[checkpoint_callback])
    
    dm = EmotionDataModule(train_dir, test_dir, config['batch_size'])
    dm.setup(stage='fit')
    
    trainer.fit(model, dm)

In [8]:
import torchmetrics
from typing import Counter
from torchvision.datasets import ImageFolder
from sklearn.utils.class_weight import compute_class_weight
dataset = ImageFolder(train_dir)
class_counts = dict(Counter(dataset.targets))
class_weights = compute_class_weight('balanced', classes=np.unique(dataset.targets), y=dataset.targets)
class_weights = torch.tensor(class_weights, dtype=torch.float32)


In [9]:

run_train(config)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3050 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\loggers\wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:652: Checkpoint directory D:\projects\face-emotion-tracker\models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:475: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


d:\projects\face-emotion-tracker\venv\Lib\site-packages\torchmetrics\utilities\prints.py:43: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)  # noqa: B028
d:\projects\face-emotion-tracker\venv\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:419: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


Epoch 2: 100%|██████████| 449/449 [01:31<00:00,  4.93it/s, v_num=64kp]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 449/449 [01:31<00:00,  4.91it/s, v_num=64kp]
